In [16]:
import os, sys
import os.path as osp
sys.path.append(osp.join("..", "yolo_tracker"))
from darknet import Darknet
from __future__ import division
import time
import torch
import torch.nn as nn
from torch.autograd import Variable
import numpy as np
import cv2
from util import *
import os
import os.path as osp
from darknet import Darknet
from preprocess import prep_image, inp_to_image
import pandas as pd
import random
import pickle as pkl
import itertools
from args import arg_parse
from detector import predict
from postprocess import postprocess

In [19]:
# usefull class if args are not available via terminal
class Args:
    def __init__(self):
        self.bs = 1
        self.confidence = .5
        self.cfgfile = "cfg/yolov3.cfg"
        self.nms_thresh = .4
        self.weightsfile = "yolov3.weights"
        self.images = None
        self.reso = "416"
        self.scales = "1,2,3"
        self.saveto = ""
        self.silent = None
        self.cuda = "3"
        self.det = "det"
        self.vot = "../../../vot2016/"
        self.pp = "mfc"
        self.saveto = "results"

In [10]:
def fill_zeros(folder):
    return [[0, 0, 0, 0] for i in os.listdir(folder) if i.endswith(".jpg")]

In [11]:
def do_full_postprop(predictions, postprocessor, vot_path):

    RESULTS = predictions["results"]
    LENGTH = predictions["length"]
    CUDA = predictions["CUDA"]

    res = dict()

    for im_class in RESULTS:
        if postprocessor is not None:
            pp_result = postprocess((RESULTS[im_class], LENGTH, CUDA), osp.join(vot_path, im_class), postprocessor)
        if len(pp_result) != 0:
            res[im_class] = pp_result[:, 1:5].tolist()            
        else:
            res[im_class] = fill_zeros(osp.join(vot_path, im_class))

        return res

In [12]:
def save(res, folder):
    if not os.path.exists(folder):
        os.makedirs(folder)
    os.chdir(folder)
    for name in res:
        print("\n".join([",".join([str(a) for a in x])
                         for x in res[name]]), file=open(name + ".txt", 'w+'))

In [21]:
# args = arg_parse()
os.chdir("../yolo_tracker")
args = Args()
vot_path = args.vot
saveto = args.saveto
pp = None if args.pp.lower() == "none" else args.pp

In [4]:
predictions = predict(args)

Loading network.....
Network successfully loaded
processing bolt1
/mnt/home.stud/zabulvol/sshfs/YOLO-tracker/yolo_tracker/../../../vot2016/bolt1/00000001.jpg
00000001.jpg         predicted in  0.033 seconds
Objects Detected:    person person person person person person person person person sports ball sports ball sports ball
----------------------------------------------------------
/mnt/home.stud/zabulvol/sshfs/YOLO-tracker/yolo_tracker/../../../vot2016/bolt1/00000002.jpg
00000002.jpg         predicted in  0.035 seconds
Objects Detected:    person person person person person person person person person person person person person person person person sports ball sports ball sports ball sports ball
----------------------------------------------------------
/mnt/home.stud/zabulvol/sshfs/YOLO-tracker/yolo_tracker/../../../vot2016/bolt1/00000003.jpg
00000003.jpg         predicted in  0.029 seconds
Objects Detected:    person person person person person person person person person person p

In [17]:
predictions = do_full_postprop(predictions, pp, vot_path)

In [18]:
save(predictions, saveto)